In [1]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import utils
import json
import os
import git
import warnings
warnings.filterwarnings("ignore")

In [2]:
from dotenv import load_dotenv
load_dotenv()
HOPSWORKS_API_KEY = os.getenv("HOPSWORKS_API_KEY")
os.environ["HOPSWORKS_API_KEY"] = HOPSWORKS_API_KEY

## Get the necessary data from Hopsworks

In [3]:
project = hopsworks.login()
fs = project.get_feature_store() 
# secrets = utils.secrets_api(project.name)

CITY = "dublin"
STATION = "HEUSTON BRIDGE (NORTH)"

# latitude =
# longitude =

today = datetime.datetime.now()

2025-01-11 09:00:21,686 INFO: Initializing external client


2025-01-11 09:00:21,689 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-01-11 09:00:23,583 INFO: Python Engine initialized.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1207494


In [4]:
# Retrieve feature groups
bike_fg = fs.get_feature_group(
    name='bike_data',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather_data',
    version=1,
)


## Clone and pull the repository with the bike data

In [5]:
# Configuration
REPO_URL = "https://github.com/MaxHalford/bike-sharing-history"
CLONE_DIR = "./bike_data/bike-sharing-history"
TARGET_CITY = "dublin"
FILE_NAME = "jcdecaux.geojson"

In [6]:
if not os.path.exists(CLONE_DIR):
    print("Cloning repository...")
    git.Repo.clone_from(REPO_URL, CLONE_DIR)
repo = git.Repo(CLONE_DIR)

# go to main and pull the latest changes
repo.git.checkout("main", force=True)
repo.remotes.origin.pull()

## Get the latest datetime present in the bike data

In [7]:
# last_bike_datetime = "2025-01-06 15:06:11 UTC"

bike_df = bike_fg.read()

last_bike_datetime = bike_df["datetime"].max()
last_bike_datetime = last_bike_datetime.strftime("%Y-%m-%d %H:%M:%S %Z")

last_bike_datetime

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.53s) 


'2025-01-09 23:49:15 UTC'

## Loop through the commits and convert the bike data into a dataframe

In [8]:
results = {}

# Populate the results dict with the stations
data_file = os.path.join(CLONE_DIR, "data/stations/" + TARGET_CITY + "/" + FILE_NAME)
if os.path.exists(data_file):
    with open(data_file, "r") as f:
        data = f.read()
        data = json.loads(data)
        for feature in data["features"]:
            name = feature["properties"]["name"]
            results[name] = []

In [9]:
start_time = datetime.datetime.now()
last_day_and_hour = None

for commit in repo.iter_commits():
    # Stop when we reach the earliest bike date
    if commit.committed_datetime <= datetime.datetime.strptime(last_bike_datetime, "%Y-%m-%d %H:%M:%S %Z").replace(tzinfo=datetime.timezone.utc):
        print("breaking at: ", commit.committed_datetime)
        break

    # Skip commits from today
    if commit.committed_datetime > today.replace(tzinfo=datetime.timezone.utc).replace(hour=0, minute=0, second=0, microsecond=0):
        first_commited_datetime = repo.commit().committed_datetime
        continue

    day_and_hour = commit.committed_datetime.replace(minute=0, second=0, microsecond=0)
    # print("day_and_hour: ", day_and_hour, " - last_day_and_hour: ", last_day_and_hour)
    if day_and_hour == last_day_and_hour:
        continue
    last_day_and_hour = day_and_hour

    # Get the data for the commit
    print("Processing commit: ", commit.hexsha, " - ", commit.committed_datetime)
    try:
        repo.git.checkout(commit.hexsha, force=True)
    except Exception as e:
        print("Error checking out commit: ", e)
        break

    data_file = os.path.join(CLONE_DIR, "data/stations/" + TARGET_CITY + "/" + FILE_NAME)
    if os.path.exists(data_file):
        with open(data_file, "r") as f:
            data = f.read()
            data = json.loads(data)
            for feature in data["features"]:
                try:
                    results[feature["properties"]["name"]].append([feature["properties"]["available_bikes"], commit.committed_datetime])
                except KeyError:
                    continue

print(results)


Processing commit:  bf81b98ac4c9e4966b44117c7cd2e00068139286  -  2025-01-10 23:49:13+00:00


Processing commit:  5e85ab4a288fae937a4bfc9c8b28ebd648a3a768  -  2025-01-10 22:49:06+00:00


Processing commit:  4fd12731e66ffbd0bd632ff50f459b9ab4c45cf0  -  2025-01-10 21:49:00+00:00


Processing commit:  3adfc4b3dee6b847e1ac8644bc83d5c7a1698630  -  2025-01-10 20:49:07+00:00


Processing commit:  9cd86cdf2de9b751edc5362677c2dc448130bf0a  -  2025-01-10 19:49:07+00:00


...

Processing commit:  7ac96475e5467e4bd6a2175149f002994726e91b  -  2025-01-10 03:49:22+00:00


Processing commit:  aee94f4904f56a0821cbe76dd258e87f5788f0d2  -  2025-01-10 02:41:49+00:00


Processing commit:  aaf0d5b874eb61094bd5b3fbbb49ac15a1eb4278  -  2025-01-10 01:33:48+00:00


Processing commit:  54054d52a44d27d5b40891007176ee220f1d4efd  -  2025-01-10 00:49:52+00:00


breaking at:  2025-01-09 23:49:15+00:00
{'CLARENDON ROW': [[22, datetime.datetime(2025, 1, 10, 23, 49, 13, tzinfo=<git.objects.util.tzoffset object at 0x00000264B0FFF3A0>)], [20, datetime.datetime(2025, 1, 10, 22, 49, 6, tzinfo=<git.objects.util.tzoffset object at 0x00000264B109A740>)], [19, datetime.datetime(2025, 1, 10, 21, 49, tzinfo=<git.objects.util.tzoffset object at 0x00000264B109BBE0>)], [20, datetime.datetime(2025, 1, 10, 20, 49, 7, tzinfo=<git.objects.util.tzoffset object at 0x00000264B0FFF580>)], [19, datetime.datetime(2025, 1, 10, 19, 49, 7, tzinfo=<git.objects.util.tzoffset object at 0x00000264B10B66E0>)], [18, datetime.datetime(2025, 1, 10, 18, 49, 34, tzinfo=<git.objects.util.tzoffset object at 0x00000264B10BFCA0>)], [22, datetime.datetime(2025, 1, 10, 17, 49, 2, tzinfo=<git.objects.util.tzoffset object at 0x00000264B109BC40>)], [15, datetime.datetime(2025, 1, 10, 16, 49, 48, tzinfo=<git.objects.util.tzoffset object at 0x00000264B1206860>)], [15, datetime.datetime(2025, 

In [10]:
# turn results into a dataframe
df_bike_today = pd.DataFrame()

for station, values in results.items():
    if len(values) > 0:
        df = pd.DataFrame(values, columns=["num_bikes_available", "datetime"])
        df["datetime"] = pd.to_datetime(df["datetime"], utc=True)
        df["station"] = station.replace(" ", "_")
        df_bike_today = pd.concat([df_bike_today, df])

# if empty, do nothing
if not df_bike_today.empty:
    df_bike_today.dropna(inplace=True)
    df_bike_today["num_bikes_available"] = df_bike_today["num_bikes_available"].astype("float32")
    df_bike_today = df_bike_today[df_bike_today['station'].isin([STATION.replace(" ", "_")])]
    df_bike_today


## Fetch the weather data for the same time period

In [11]:
forecast_df = utils.get_hourly_weather_forecast(CITY)
forecast_df = forecast_df.rename(columns={'date_x': 'datetime'})
forecast_df = forecast_df.drop(columns=['date_y', 'date_only'])
forecast_df.dropna(inplace=True)

print(forecast_df.empty)

forecast_df

features: {'hourly': ['temperature_2m', 'apparent_temperature', 'rain', 'snowfall', 'wind_speed_10m'], 'daily': ['daylight_duration', 'rain_sum']}
params: {'latitude': 53.35, 'longitude': -6.26, 'hourly': ['temperature_2m', 'apparent_temperature', 'rain', 'snowfall', 'wind_speed_10m'], 'daily': ['daylight_duration', 'rain_sum']}
Coordinates 53.5°N -6.25°E
Elevation 11.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
False


,datetime,temperature_2m,apparent_temperature,rain,snowfall,wind_speed_10m,daylight_duration,rain_sum,city
0,2025-01-11 00:00:00+00:00,5.35,0.736369,0.0,0.0,18.899143,28413.685547,3.0,dublin
1,2025-01-11 01:00:00+00:00,5.20,0.613584,0.2,0.0,19.174856,28413.685547,3.0,dublin
2,2025-01-11 02:00:00+00:00,5.05,0.475718,0.2,0.0,19.959721,28413.685547,3.0,dublin
3,2025-01-11 03:00:00+00:00,5.00,0.464148,0.2,0.0,20.466246,28413.685547,3.0,dublin
4,2025-01-11 04:00:00+00:00,5.05,0.436636,0.5,0.0,21.479664,28413.685547,3.0,dublin
...,...,...,...,...,...,...,...,...,...
235,2025-01-20 19:00:00+00:00,4.85,3.185979,0.0,0.0,1.609969,29881.912109,0.0,dublin
236,2025-01-20 20:00:00+00:00,4.90,3.245836,0.0,0.0,1.609969,29881.912109,0.0,dublin
237,2025-01-20 21:00:00+00:00,5.05,3.308195,0.0,0.0,2.414953,29881.912109,0.0,dublin
238,2025-01-20 22:00:00+00:00,5.30,3.412459,0.0,0.0,3.758510,29881.912109,0.0,dublin


## Insert the bike and weather data into Hopsworks

In [12]:
if df_bike_today.empty:
    print("No bike data available for today")
else:
    bike_fg.insert(df_bike_today)

Uploading Dataframe: 0.00% |                                      | Rows 0/24 | Elapsed Time: 00:00 | Remaining Time: ?

Uploading Dataframe: 83.33% |██████████████████████████▋     | Rows 20/24 | Elapsed Time: 00:01 | Remaining Time: 00:00

Uploading Dataframe: 100.00% |███████████████████████████████| Rows 24/24 | Elapsed Time: 00:01 | Remaining Time: 00:00

Launching job: bike_data_1_offline_fg_materialization


Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1207494/jobs/named/bike_data_1_offline_fg_materialization/executions


In [13]:
if forecast_df.empty:
    print("No weather forecast available for today")
else:
    weather_fg.insert(forecast_df, write_options={"wait_for_job": True})


Uploading Dataframe: 0.00% |                                     | Rows 0/240 | Elapsed Time: 00:00 | Remaining Time: ?

Uploading Dataframe: 100.00% |█████████████████████████████| Rows 240/240 | Elapsed Time: 00:00 | Remaining Time: 00:00

Launching job: weather_data_1_offline_fg_materialization


Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1207494/jobs/named/weather_data_1_offline_fg_materialization/executions


2025-01-11 09:01:27,022 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED


2025-01-11 09:01:30,218 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED


2025-01-11 09:02:49,786 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED


2025-01-11 09:02:49,938 INFO: Waiting for log aggregation to finish.


2025-01-11 09:03:01,886 INFO: Execution finished successfully.
